In [ ]:
import numpy as np
from scipy import misc, ndimage, spatial
from scipy.ndimage import morphology

from skimage import io, img_as_ubyte, img_as_float
from skimage.transform import rescale, hough_circle, hough_circle_peaks
from skimage.color import rgb2lab, lab2rgb, rgba2rgb
from skimage.io import imread, imsave, imshow, show, imshow_collection, imread_collection
from skimage import color, exposure, viewer, data
from skimage.util import invert, random_noise, montage
from skimage import filters, feature, morphology, segmentation
from skimage.draw import circle_perimeter
from skimage.morphology import thin

from PIL import Image, ImageFont, ImageDraw

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors
import webcolors

from copy import deepcopy

In [ ]:
#predicts one of the six common iris colours from the average intensity of the iris.
def predict_eye_color(rgbval):
    con1,con2,con3 = 0 < np.abs(rgbval[0]-rgbval[1]) < 15,0 < np.abs(rgbval[1]-rgbval[2]) < 15,0 < np.abs(rgbval[0]-rgbval[2]) < 15
    con4 = rgbval[0] > 100 and rgbval[1] > 100 and rgbval[2] > 100
    if con1 and con2  and con3 and con4: #if gray is likely
        return "gray"
    elif rgbval[2] > rgbval[1] and rgbval[2] > rgbval[0]: #if blue component greater than red and green compeonents
        return "blue"
    elif rgbval[1] > rgbval[0] and rgbval[1] > rgbval[2]: #if green component is greater than the other two
        return "green"
    else: #if red component is greatest
        if rgbval[2] < 10 and rgbval[0] > 150:
            return "amber"
        elif rgbval[0] < 100 or rgbval[2] < 60 and rgbval[1] < 80:
            return "brown"
        else:
            return "hazel"

In [ ]:
#preprocesses an image by performing dilation and removing unnecessary noise in the image
def dilation_preprocess(img_th):
    dilated_img = deepcopy(img_th)
    for i in range(2):
        dilated_img = morphology.binary_dilation(dilated_img)

    coords_dil = np.where(dilated_img == 1)
    average = np.average(coords_dil,1)

    #calculate center pixel and distance from center to the average position of all foreground pixels
    center = np.array([img_th.shape[0]/2,img_th.shape[1]/2])
    center_radius = np.abs(center[1]-average[1])
    center_radius_x = np.abs(center[0]-average[0])

    if center_radius < 10: center_radius = 0.2*img_th.shape[1]
    if center_radius_x < 10: center_radius_x = 0.2*img_th.shape[0]

    #calculate a point we want to focus on
    turning_point = (center+average)/2

    #remove noise on the sides of the image
    for x in range(img_th.shape[0]):
        for y in range(img_th.shape[1]):
            if (x < turning_point[0] - 2.5*center_radius_x) or (x > turning_point[0] + 2.5*center_radius_x):
                dilated_img[x][y] = 0
            if (y < turning_point[1] - 2.5*center_radius) or (y > turning_point[1] + 2.5*center_radius):
                dilated_img[x][y] = 0
    
    return dilated_img       

In [ ]:
#binary thresholding according to some threshold value th
def get_threshold_image(im,th):
    return im < th

In [ ]:
#perform circle hit-or-miss on a thresholded image to find pupil
def erosion_preprocess(dilated_img):
    disk = morphology.disk(2)
    eroded = dilated_img
    coords = np.where(eroded == 1)
    while len(coords[0]) > 500:
        eroded = morphology.binary_erosion(eroded,disk)
        coords = np.where(eroded == 1)

    return coords

In [ ]:
#find the iris center using canny edge detection and hough circles
def get_iris_center(eye_threshold,eye_img): 
    eyeEdges = feature.canny(eye_threshold)
    a = eye_img.shape[0]
    b = eye_img.shape[1]
    houghRadii = np.arange(int(a/6),int(a/2), 1)
    houghRes = hough_circle(eyeEdges, houghRadii)
    accums, cx, cy, radii = hough_circle_peaks(houghRes, houghRadii, total_num_peaks=1)

    return cx,cy,radii

In [ ]:
#find approximate position of pupil
def calculate_pupil_position(test_img,gray_noiseless,curr_th):
    group_a = []
    group_b = []
    num_coords = len(group_a)+len(group_b)

    #performs this again with a higher threshold in case it does not find anything
    while num_coords == 0:
        img_th = get_threshold_image(gray_noiseless,curr_th)
        dilated_img = dilation_preprocess(img_th)
        coords = erosion_preprocess(dilated_img)

        for r in range(len(coords[0])):
            p = np.array([coords[0][r],coords[1][r]])
            if len(group_a) == 0:
                group_a.append(p)
            elif np.linalg.norm(np.average(group_a)-p) >= 0.25*test_img.shape[1]:
                group_b.append(p)
            else:
                group_a.append(p)

        num_coords = len(group_a)+len(group_b)
        curr_th += 0.05
           
    radx = int(0.12*test_img.shape[0])
    rady = int(0.12*test_img.shape[1])

    a_avg_intensity = 0
    for acoord in group_a:
        a_avg_intensity += gray_noiseless[acoord[0]][acoord[1]]

    b_avg_intensity = 0
    for bcoord in group_b:
        b_avg_intensity += gray_noiseless[bcoord[0]][bcoord[1]]

    if len(group_a) == 0:
        avg_a = 2000
    else:
        avg_a = a_avg_intensity/len(group_a)
    if len(group_b) == 0:
        avg_b = 2000
    else:
        avg_b = b_avg_intensity/len(group_b)

    #xavg = int(np.floor(np.average(coords[0])))
    #yavg = int(np.floor(np.average(coords[1])))
    #print(len(group_a),len(group_b))
    if avg_b >avg_a:
        xavg,yavg = np.average(group_a,0)
    else:
        xavg,yavg = np.average(group_b,0)
    xavg = int(np.floor(xavg))
    yavg = int(np.floor(yavg))

    #print(xavg,yavg,radx,rady)

    coord1,coord2,coord3,coord4 = xavg-radx,xavg+radx,yavg-rady,yavg+rady
    if coord1 < 0: coord1 = 0
    if coord2 >= test_img.shape[0]: coord2 = test_img.shape[0]-1
    if coord3 < 0: coord3 = 0
    if coord4 >= test_img.shape[1]: coord4 = test_img.shape[1]-1

    return coord1,coord2,coord3,coord4,xavg,yavg,radx,rady

In [ ]:
#call this as the main function that returns the predicted colour
#INPUT: the file adress and name, example: "brown/brown1.jpg"
def main(image_file):
    #read in input image and convert to gray image then apply smoothing
    im = Image.open(image_file)
    test_img = np.asarray(im)
    gray_img = color.rgb2gray(test_img)
    mean = filters.threshold_minimum(gray_img)
    gray_noiseless = filters.gaussian(gray_img, 1)

    #calculate a threshold intensity
    min_intensity = np.min(gray_noiseless)
    if min_intensity < 0.1:
        curr_th =  0.05 + 2*min_intensity
    else:
        curr_th = 0.05 + min_intensity
    coord1,coord2,coord3,coord4,xavg,yavg,radx,rady = calculate_pupil_position(test_img,gray_noiseless,curr_th)
    
    #after getting a sliced image of the pupil
    eye_img = test_img[coord1:coord2,coord3:coord4]
    gray_eye = gray_img[coord1:coord2,coord3:coord4]
    #eye_threshold = gray_threshold[xavg-100:xavg+100,yavg-100:yavg+100]
    eye_noiseless = gray_noiseless[coord1:coord2,coord3:coord4]
    eye_threshold = np.zeros(gray_eye.shape)
    for i in range(gray_eye.shape[0]):
        for j in range(gray_eye.shape[1]):
            if eye_noiseless[i][j] > curr_th:
                eye_threshold[i][j] = 1

    #get center and radius of circle from canny edge and hough circle detection
    cx,cy,radii = get_iris_center(eye_threshold,eye_img)

    greatest_circle = 0
    cxt = cy[greatest_circle] + xavg - radx
    cyt = cx[greatest_circle] + yavg - rady

    radi = radii[greatest_circle]*(test_img.shape[1]/eye_img.shape[1])

    iris_img = deepcopy(test_img)
    for i in range(test_img.shape[0]):
        for j in range(test_img.shape[1]):
            if np.linalg.norm(np.array([i,j])-np.array([cxt,cyt])) >= 0.6*radi:
                iris_img[i][j] = [0,0,0]


    gray_iris = color.rgb2gray(iris_img)
    #plt.imshow(gray_iris,cmap=plt.cm.gray)

    eye_threshold = deepcopy(gray_iris)
    for i in range(test_img.shape[0]):
        for j in range(test_img.shape[1]):
            if gray_iris[i][j] < 0.12 or gray_iris[i][j] > 0.7:
                eye_threshold[i][j] = 0
            else:
                eye_threshold[i][j] = 1

    #calculate the average colour from the segmented iris
    avg_colour = np.array([0,0,0])
    count = 0
    output_img = deepcopy(test_img)
    for i in range(test_img.shape[0]):
        for j in range(test_img.shape[1]):
            if (eye_threshold[i][j]):
                avg_colour += test_img[i][j]
                count += 1
            else:
                output_img[i][j] = [0,0,0]
    
    avg_colour = avg_colour/count
    avg_colour[np.isnan(avg_colour)] = 0
    colour = np.zeros((3,3,3))
    for i in range(len(avg_colour)):
        avg_colour[i] = avg_colour[i]/255
    colour[1][1] = avg_colour

    reqColour = []
    for i in range(3):
        reqColour.append(int(np.floor(avg_colour[i]*256)))

    #predict colour from average intensity and return result
    out_colour = predict_eye_color(reqColour)
    #print(out_colour) <---- IF YOU WANT TO PRINT OUT THE COLOUR WHEN RUNNING THROUGH ALL THE IMAGES
    return out_colour, output_img

In [ ]:
#EXAMPLE of running this script
col_result,im_result = main("images/brown/brown3.jpg")
print(col_result)
io.imshow(im_result)

In [ ]:
#This is the details of the image data set needed if you want to run the below code
num_colours = 6
colours = ['amber', 'blue', 'brown', 'gray', 'green', 'hazel']
num_images = [11,11,20,18,18,18]
num_correct_predictions = [0,0,0,0,0,0]
results = []

In [ ]:
#This calls the main function for all images in the dataset and returns the output segmented iris image as well as the predicted colour
for c in range(len(colours)):
    temp = []
    for j in range(1,num_images[c]+1):
        #print(colours[c]+"/"+colours[c]+str(j)+".jpg")
        temp.append(main("images/"+colours[c]+"/"+colours[c]+str(j)+".jpg"))
        if temp[0] == colours[c]:
            num_correct_predictions[c] += 1
    results.append(temp)

In [ ]:
#Used for obtaining stats
num_correct_predictions = [1,11,17,1,5,13]
for j in range(6):
    print(colours[j],"\ttotal number images: ", num_images[j], "\ttotal predicted correct: ", num_correct_predictions[j],"\tsuccess rate: ", (num_correct_predictions[j]/num_images[j])*100)

totalcorrect = np.sum(num_correct_predictions)
total = np.sum(num_images)
print("total success rate: ",totalcorrect/total)